<a href="https://colab.research.google.com/github/carmenfontenla/DEEP_LEARNING_4_UFV/blob/main/Canciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GENERACIÓN DE TEXTO

Hecho por Carmen Fontenla Cambón y Alegría Gimeno Arnau.

Un generador de texto es un modelo de red neuronal que se utiliza para crear y desarrollar textos. En este caso, se utilizarán dos archivos de texto, uno que contiene las Top 100 canciones de Spotify del momento en español, y otro que posee 50 canciones del grupo Estopa. El objetivo de esta práctica es crear la letra de un verso de una canción a partir de las anteriores. Para ello hay que entrenar el programa para que detecte cuáles son los patrones de palabras que se encuentran en las canciones para generar letras nuevas.

In [ ]:
import numpy as np
# original_distribution es una matriz 1D NumPy de valores de
# probabilidad que deben sumar 1. La temperatura es un factor
# que cuantifica la entropía de la distribución de salida.
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    # Devuelve una versión reponderada de la distribución original.
    # Es posible que la suma de la distribución ya no sea 1, por lo
    # que la dividiremos por su suma para obtener la nueva distribución
    return distribution / np.sum(distribution)

Genera una distribuccion de temperaturas, es decir,  probabilidades de escoger ciertas palabras o en este caso las palabras mas comunes. Si la temperatura es baja, las frases que se crearan seran muy predecibles y habra muy poca diferencia entre las distintas epochs de entrenamiento. Sin embargo,si las temperaturas son altas dan como resultado distribuciones de mayor entropia, que generan datos mas sorprendentes y no estructurados. Este código se implementa antes de los archivos, ya que es parte de la arquitectura de la red.

DATASET 1: REGGAETON

Vamos a empezar por el archivo de texto de las top 100 canciones de Reggaeton en España, el cual está contenido en el archivo 'SUPERNOVA SAIKO.txt'.

Antes de entrenar la red neuronal, vamos a realizar un preprocesamiento de la bbdd.

Vamos a crear un archivo donde cada estrofa (cada 4 versos) se considerara un .txt de entrenamiento y las guardadremos dentro de una carperta llamada estrofas.

In [ ]:
import os #creamos la carpeta estrofas
os.mkdir("/content/estrofas/")

Para comenzar guardadermos todos las estrofas en una lista, luego borraremos los nombres de las canciones que estan escritos en mayúsculas y por ultimo borraremos \n que aparece al final de cada verso.

In [ ]:

with open ("/content/SUPERNOVA SAIKO.txt") as archivo:
  lineas=archivo.readlines()
print(lineas)
lineaspre=list()
for i in lineas:
  lineaspre.append(i[:-2])
print(lineaspre)
lineaspre2=list()
for i in lineaspre:
  if i.isupper():
    continue
  else:
    lineaspre2.append(i)
print(lineaspre2)
# Nótese que también se quitaron la última letra de cada frase. Este error proviene de la segunda lista, generada por el siguiente comando lineaspre.append(i[:-2]) y, dado que el código tardaba
# en ejecutarse aproximadamente 9 horas y se tuvo que recurrir a una máquina prestada, así que fue imposible por el poco tiempo de margen que teníamos para volver a generar las canciones
# sin el error. Este error se solucionó para el segundo dataset.

['SUPERNOVA SAIKO \n', 'Siempre me ha gustado como suenan las palabras\n', 'en el idioma del amor\n', 'Nunca me ha gustado como suenan las palabras\n', 'en el idioma del rencor\n', '\n', 'No me quiero ir de aquí sin probar\n', 'que se siente estar a flor de piel\n', 'no me quiero ir de aquí sin probar\n', 'la miel amarga del fracaso\n', '\n', 'Estoy esperando el momento para explotar\n', 'quiero ser una supernova\n', 'y brillar para siempre\n', '\n', 'Siempre he sentido que estoy fuera de lugar\n', 'en un mundo sin corazón\n', 'Nunca he sabido como encajar\n', 'en este mundo de ilusión\n', '\n', 'No me quiero ir de aquí sin probar\n', 'que se siente estar a flor de piel\n', 'no me quiero ir de aquí sin probar\n', 'la miel amarga del fracaso\n', '\n', 'Estoy esperando el momento para explotar\n', 'quiero ser una supernova\n', 'y brillar para siempre\n', '\n', '¿Por qué no puedo ser feliz con lo que tengo?\n', '¿Por qué siempre quiero más?\n', '¿Por qué no puedo vivir sin hacerme daño?\n

Ahora una vez que ya hemos completado este proceso, vamos a guardar dentro de archivos .txt cada estrofa. Para ello necesitamos calcular la longitud de las lineaspre2 y dividirlas entre 4 ya que queremos cada estrofa en uno distinto.

In [ ]:
path="/content/estrofas/"
numestrofas=int(len(lineaspre2)/4)
print(numestrofas)

1488


Ahora vamos a correr el for para generar los archivos de texto y que guardaremos en la carpeta 'estrofas'.

In [ ]:
for i in range(numestrofas):
  nombre=path+"letras"+str(i)+".txt"
  f=open(nombre,"a")
  f.write(lineaspre2[4*i]+" "+lineaspre2[4*i+1]+" "+lineaspre2[4*i+2]+" "+lineaspre2[4*i+3])


Creamos un dataset en el que trabajará la red neuronal,con la herramienta text_dataset_from_directory, el cual nos permitirá etiquetar cada archivo como perteneciente a una misma clase.

In [ ]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="estrofas", label_mode=None, batch_size=256)


Found 1488 files belonging to 1 classes.


Antes de hacer la vectorizacion del texto, necesitamos una lista de cuantas palabras unicas hay, para saber el tamaño del vocaulario que tenemos.
Primero creamos una lista con todas las palabras y luego creamos otra lista con las palabras unicas, sin que haya repeticiones.

In [ ]:
palabras=list()
for i in range(len(lineaspre2)):
  for j in lineaspre2[i].split():
    palabras.append(j)
print(palabras)

['Siempre', 'me', 'ha', 'gustado', 'como', 'suenan', 'las', 'palabra', 'en', 'el', 'idioma', 'del', 'amo', 'Nunca', 'me', 'ha', 'gustado', 'como', 'suenan', 'las', 'palabra', 'en', 'el', 'idioma', 'del', 'renco', 'No', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'que', 'se', 'siente', 'estar', 'a', 'flor', 'de', 'pie', 'no', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'la', 'miel', 'amarga', 'del', 'fracas', 'Estoy', 'esperando', 'el', 'momento', 'para', 'explota', 'quiero', 'ser', 'una', 'supernov', 'y', 'brillar', 'para', 'siempr', 'Siempre', 'he', 'sentido', 'que', 'estoy', 'fuera', 'de', 'luga', 'en', 'un', 'mundo', 'sin', 'corazó', 'Nunca', 'he', 'sabido', 'como', 'encaja', 'en', 'este', 'mundo', 'de', 'ilusió', 'No', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'que', 'se', 'siente', 'estar', 'a', 'flor', 'de', 'pie', 'no', 'me', 'quiero', 'ir', 'de', 'aquí', 'sin', 'proba', 'la', 'miel', 'amarga', 'del', 'fracas', 'Estoy', 'esperando', 'el', 'momento', 'pa

In [ ]:
unicas=list(set(palabras))
print(len(unicas))
print(len(palabras))


6621
38600


Como podemos ver de las 38600 palabras que tenemos en total, solo hay 6621 que no estan repetidas, es decir, palabras unicas, estas pasaran a formar parte del vocabulario.

Esta parte vectoriza el texto, eso significa que tranforma una frase en un vector de numeros, cada entrada del vector simboliza una palabra, por eso es importante saber con cuantas palabras contamos.

In [ ]:
from tensorflow.keras.layers import TextVectorization
#Nuetsras secuencia mas larga tendra una longitud de 100 palabras
sequence_length = 100
#Decidimos trabajar co 2/3 del vocabulario total
vocab_size = 4414
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    # Queremos devolver secuencias de índice
    # de palabras con valores enteros
    output_mode="int",
    # Trabajaremos con entradas y objetivos de longitud
    # 100 (pero como compensaremos los objetivos en 1,
    # el modelo verá secuencias de longitud 99).
    output_sequence_length=sequence_length,
)



text_vectorization.adapt(dataset)

In [ ]:
#función que prepara el dataset, es la continuación de la vectorización de los textos.
def prepare_lm_dataset(text_batch):

    # Convertimos un lote de textos (cadenas)
    # en un lote de secuencias enteras.
    vectorized_sequences = text_vectorization(text_batch)
    # Creamos entradas cortando la última palabra de las secuencias
    x = vectorized_sequences[:, :-1]
    # Creamos objetivos compensando las secuencias por 1.
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

Creamos dos clases distintas. La primera 'PositionalEmbedding(layers.Layer)', que nos permite hacer un incruste o encaje, en el contexto de este trabajo nos va a permitir mapear objetos o palabras con significado similar a puntos cercanos, es decir, hace búsqueda en la vecindad de una palabra, es decir, qué palabras suelen estar antes o después. Por otro lado, 'TransformerDecoder(layers.Layer)' nos va a permitir decodificar los vectores a palabras.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

A continuación se establecen los parámetros que se usarán para generar los nuevos versos.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 2048
num_heads = 2
# De ejemplo
sequence_length = 100
vocab_size = 4414

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
# Softmax sobre posibles palabras de vocabulario,
# calculado para cada timestep de secuencia de salida.
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

Se va a utilizar un callback para generar texto usando diferentes rangos de temperatura después de cada epoch, con el fin de evaluar cómo evoluciona el texto generado y el impacto de la temperatura en la estrategia de muestreo. El texto generado se iniciará con el indicador "El amor" para todas las generaciones.

In [ ]:
import numpy as np

# Dict (diccionario) que asigna o mapea índices de palabras
# a cadenas, para usarlos para decodificar texto
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

# Implementa el muestreo de variable-temperatura
# a partir de una distribución de probabilidad
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 # Indicación o prompt que usamos para
                 # iniciar la generación de texto
                 prompt,
                 # Cuantas palabras generar
                 generate_length,
                 model_input_length,
                 # Rango de temperaturas
                 # a usar para el muestreo
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
        vectorized_prompt = text_vectorization([prompt])[0].numpy()
        self.prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            # Al generar texto, comenzamos
            # desde nuestro prompt o aviso.
            sentence = self.prompt
            for i in range(self.generate_length):
                # Alimentamos nuestro modelo con la secuencia actual
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                # Recuperamos las predicciones del último timestep y las
                # utilizamos para muestrear o probar una nueva palabra.
                next_token = sample_next(
                    # O [0, i, :]
                    predictions[0, self.prompt_length - 1 + i, :]
                )
                sampled_token = tokens_index[next_token]
                # Agregamos la nueva palabra a
                # la secuencia actual y repetimos
                sentence += " " + sampled_token
            print(sentence)

prompt = "El amor"
text_gen_callback = TextGenerator(
    prompt,
    #Es importante que la longitud de la secuencia sea menor que sequence_length que se establecio en la vectorizacion
    generate_length=40,
    model_input_length=sequence_length,
    # Usaremos un rango diverso de
    # temperaturas para muestrear texto,
    # para demostrar el efecto de la
    # temperatura en la generación de texto.
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [ ]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200
6/6 [==============================] - ETA: 0s - loss: 7.5342 == Generating with temperature 0.2
El amor me tú si pasao fetuccin ando cail lo estás no arrepientas a dudas te extraño zaitut fingiendo respirar quedaban yú rápid glopeta este nos tos rompediscotec que fueg amore perreo parezca vuelta tratándolo la dunn 6 toqué tres verano novia
== Generating with temperature 0.5
El amor toas una las ir la que explica asustes chiquichiquichiquidopchipchipchipcha normal hacer mam que otr yuh yeahyeahyeahyeahyeahyea dud mañan llevé cabez porsche gir sío gafitas llama you esa aun lo for recuperart dosciento quier perro pero las ve perdiendo saltemos dejan
== Generating with temperature 0.7
El amor ver llenaba rojos solos darme vieran gusto trist voz panadol toda buenos traducto dije ti rollsroyc que soy volvió iré es cuarto list si baño party caraj dicho oficial brat zeta arranque placere abra leve tu que  jodiste escribiste
== Generating with temperature 1.0
El amor señal la porqu

Como conclusion podemos observar que, tras comparar los textos generados en distintas epochs (principio medio y final) y a distintas temperaturas. Sin embargo, la letra generada no tiene mucho sentido, ya que hay palabras que no tienen sentido dentro del contexto. Sin embargo, las redes neuranles como complemento del proceso creativo están muy bien, ya que son una herramienta que ayuda a las personas a que sea más fácil generar algo, pero si añadimos a esto el dilema moral que se ha generado por plagio tras este tipo de programas, podemos ver que al ser tan fácil que una máquina genere el texto por una persona, se va a dejar de pensar y de hacer cosas por uno mismo, e incluso se van a volver letras repetitivas, al no generar canciones nuevas desde cero que amplían los dataset con los que trabajan las redes neuronales.

CAMBIAMOS DE DATASET: ESTOPA

Hemos decidido añadir un nuevo dataset, con el fin de poder mejorar el text_vectorization y asi crear frases más congruentes, ya que el reggaeton es un estilo musical, cuyas letras no son propias del habla en castellano y son letras que tienden a ser desordenadas o con poco sentido en general.

Ahora vamos a hacer este mismo proceso pero con canciones en castellano, lo cual puede ser que tenga mucho más sentido que con canciones de reggaeton.

Para el preprocesamiento de este nuevo dataset, hemos tenido que quitar la palabra Estopa, que aparecia justo debajo de cada título de cada canción. El resto del preprocesamiento ha sido el mismo, arreglando el error de quitar la última letra de cada verso.

In [ ]:

os.mkdir("/content/estrofas2/")

In [ ]:
with open ("/content/CANCIONESESTOPA.txt") as archivo:
  lineas=archivo.readlines()
print(lineas)
lineaspre=list()
for i in lineas:
  lineaspre.append(i[:-1])
print(lineaspre)
lineaspre22=list()
for i in lineaspre:
  if i.isupper():
    continue
  else:
    lineaspre22.append(i)
lineaspre22.remove("Estopa")
print(lineaspre22)


['COMO CAMARON\n', 'Estopa\n', 'Superior a mí\n', 'Es la fuerza que me lleva\n', 'En el pulso que mantengo\n', 'Con la oscuridad que tiñen\n', 'De oscuro tus ojos negros\n', '\n', 'Y qué me cuentas del tiempo\n', 'Que pasa en tu pestañeo\n', 'Y que me trae por esta calle\n', 'De amargura y de lamento\n', '\n', 'Que yo sé que la sonrisa\n', 'Que se dibuja en mi cara\n', 'Tiene que ver con la brisa\n', 'Que abanica tu mirada\n', '\n', 'Tan despacio y tan deprisa\n', 'Tan normal y tan extraña\n', 'Yo me parto la camisa\n', 'Como Camarón\n', 'Tú me rompes las entrañas\n', 'Me trepas como una araña\n', 'Bebes del sudor que empaña\n', 'El cristal de mi habitación\n', 'Y después por la mañana\n', 'Despierto y no tengo alas\n', 'Llevo diez horas durmiendo\n', 'Y mi almohada está empapada\n', 'Todo había sido un sueño\n', 'Muy real y muy profundo\n', 'Tus ojos no tienen dueño\n', 'Porque no son de este mundo\n', '\n', 'Que no te quiero mirar\n', 'Pero es que cierro los ojos\n', 'Y hasta te veo 

In [ ]:
path="/content/estrofas2/"
numestrofas2=int(len(lineaspre22)/4)
print(numestrofas2)

387


Como podemos ver, el número de estrofas de este dataset es mucho menor que el anterior, lo que puede afectar en que el vocabulario sea más pequeño, como podremos observar a continuación.

In [ ]:
for i in range(numestrofas2):
  nombre=path+"letras"+str(i)+".txt"
  f=open(nombre,"a")
  f.write(lineaspre22[4*i]+" "+lineaspre22[4*i+1]+" "+lineaspre22[4*i+2]+" "+lineaspre22[4*i+3])

In [ ]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="estrofas2", label_mode=None, batch_size=256)


Found 387 files belonging to 1 classes.


In [ ]:
palabras2=list()
for i in range(len(lineaspre22)):
  for j in lineaspre22[i].split():
    palabras2.append(j)
print(palabras2)

['Superior', 'a', 'mí', 'Es', 'la', 'fuerza', 'que', 'me', 'lleva', 'En', 'el', 'pulso', 'que', 'mantengo', 'Con', 'la', 'oscuridad', 'que', 'tiñen', 'De', 'oscuro', 'tus', 'ojos', 'negros', 'Y', 'qué', 'me', 'cuentas', 'del', 'tiempo', 'Que', 'pasa', 'en', 'tu', 'pestañeo', 'Y', 'que', 'me', 'trae', 'por', 'esta', 'calle', 'De', 'amargura', 'y', 'de', 'lamento', 'Que', 'yo', 'sé', 'que', 'la', 'sonrisa', 'Que', 'se', 'dibuja', 'en', 'mi', 'cara', 'Tiene', 'que', 'ver', 'con', 'la', 'brisa', 'Que', 'abanica', 'tu', 'mirada', 'Tan', 'despacio', 'y', 'tan', 'deprisa', 'Tan', 'normal', 'y', 'tan', 'extraña', 'Yo', 'me', 'parto', 'la', 'camisa', 'Como', 'Camarón', 'Tú', 'me', 'rompes', 'las', 'entrañas', 'Me', 'trepas', 'como', 'una', 'araña', 'Bebes', 'del', 'sudor', 'que', 'empaña', 'El', 'cristal', 'de', 'mi', 'habitación', 'Y', 'después', 'por', 'la', 'mañana', 'Despierto', 'y', 'no', 'tengo', 'alas', 'Llevo', 'diez', 'horas', 'durmiendo', 'Y', 'mi', 'almohada', 'está', 'empapada', 'To

In [ ]:
unicas2=list(set(palabras2))
print(len(unicas2))
print(len(palabras2))


1755
7237


Nótese que de las 7237 palabras que existen en este dataset, tenemos 1755 palabras únicas, es decir, no repetidas.

In [ ]:
from tensorflow.keras.layers import TextVectorization
#Nuetsras secuencia mas larga tendra una longitud de 100 palabras
sequence_length = 80
#Decidimos trabajar co 2/3 del vocabulario total
vocab_size = 1170
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    # Queremos devolver secuencias de índice
    # de palabras con valores enteros
    output_mode="int",
    # Trabajaremos con entradas y objetivos de longitud
    # 100 (pero como compensaremos los objetivos en 1,
    # el modelo verá secuencias de longitud 99).
    output_sequence_length=sequence_length,
)



text_vectorization.adapt(dataset)

In [ ]:
def prepare_lm_dataset(text_batch):

    # Convertimos un lote de textos (cadenas)
    # en un lote de secuencias enteras.
    vectorized_sequences = text_vectorization(text_batch)
    # Creamos entradas cortando la última palabra de las secuencias
    x = vectorized_sequences[:, :-1]
    # Creamos objetivos compensando las secuencias por 1.
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 2048
num_heads = 2
# De ejemplo
sequence_length = 80
vocab_size = 1170

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
# Softmax sobre posibles palabras de vocabulario,
# calculado para cada timestep de secuencia de salida.
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

Hemos cambiado el numero de vocab_size y de sequence_lenght ya que contabamos con un numero menor de palabras.

In [ ]:
import numpy as np

# Dict (diccionario) que asigna o mapea índices de palabras
# a cadenas, para usarlos para decodificar texto
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

# Implementa el muestreo de variable-temperatura
# a partir de una distribución de probabilidad
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 # Indicación o prompt que usamos para
                 # iniciar la generación de texto
                 prompt,
                 # Cuantas palabras generar
                 generate_length,
                 model_input_length,
                 # Rango de temperaturas
                 # a usar para el muestreo
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
        vectorized_prompt = text_vectorization([prompt])[0].numpy()
        self.prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            # Al generar texto, comenzamos
            # desde nuestro prompt o aviso.
            sentence = self.prompt
            for i in range(self.generate_length):
                # Alimentamos nuestro modelo con la secuencia actual
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                # Recuperamos las predicciones del último timestep y las
                # utilizamos para muestrear o probar una nueva palabra.
                next_token = sample_next(
                    # O [0, i, :]
                    predictions[0, self.prompt_length - 1 + i, :]
                )
                sampled_token = tokens_index[next_token]
                # Agregamos la nueva palabra a
                # la secuencia actual y repetimos
                sentence += " " + sampled_token
            print(sentence)

prompt = "La fiesta"
text_gen_callback = TextGenerator(
    prompt,
    #Es importante que la longitud de la secuencia sea menor que sequence_length que se establecio en la vectorizacion
    generate_length=40,
    model_input_length=sequence_length,
    # Usaremos un rango diverso de
    # temperaturas para muestrear texto,
    # para demostrar el efecto de la
    # temperatura en la generación de texto.
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [ ]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200
2/2 [==============================] - ETA: 0s - loss: 6.9320 == Generating with temperature 0.2
La fiesta meterme sonrojando jura déjame virtual panza negras querías latiendo momento montao metes que negra cielo que o  nos olvidaron beber me vayamos papa ¡fuego manzana bar dicho nomina separan primero perderla disfruto pasando quitarse mal borrachera y boca tiñen
== Generating with temperature 0.5
La fiesta punto entra esta llega corazones efímero miestomago había es pueda falta vi caraja pocos nombre y sádico caso quiere de de juego volante quemado aliento bonito marca durmiendo pega con zalameras lugar entera esa arrepiento piernas venas ¡¿qué madero cintura
== Generating with temperature 0.7
La fiesta cacho pocas que parecía las porque salir metes llévame sangre y pistolas donde buscan dueña volandas que hablaron  cañas tu atragantarte pedal unos raja caso tocaba fuera muy callejero reza se toda primer está ruedas entendimiento demonios pensamiento de
== Generating with

Conclusiones:
- Las letras generadas con el segundo dataset tienen más sentido dentro del idioma castellano comparando la primera epoch con la 101 y la 200, vemos que el nivel de complejidad en las frases aumenta, ya que en la primera parecían palabras escritas al azar. En la 101 aparecían mini frases que tenían sentido, sin embargo, no estaban bien conectadas entre sí. Por último, en la 200, ya se puede ver una concordancia entre las frases, aunque hay que notar que existe cierta repitividad debido a las letras con las que entrenamos. Esto es bueno ya que se reproduce el estilo del artista en este caso.